In [ ]:
import pickle as pkl
import numpy as np
from numba import njit

---
---
---

In [ ]:
offsets = np.array([[-1, -1, -1], [-1, -1,  0], [-1, -1,  1],
                    [-1,  0, -1], [-1,  0,  0], [-1,  0,  1], 
                    [-1,  1, -1], [-1,  1,  0], [-1,  1,  1],
                    [ 0, -1, -1], [ 0, -1,  0], [ 0, -1,  1],
                    [ 0,  0, -1],               [ 0,  0,  1],
                    [ 0,  1, -1], [ 0,  1,  0], [ 0,  1,  1],
                    [ 1, -1, -1], [ 1, -1,  0], [ 1, -1,  1],
                    [ 1,  0, -1], [ 1,  0,  0], [ 1,  0,  1],
                    [ 1,  1, -1], [ 1,  1,  0], [ 1,  1,  1]])

In [ ]:
@njit
def isin(a, b):
    
    '''
    Checks if a list of elements is found inside a list of lists of such elements.
    
    Note: The order of the elements must be the same.
    '''
    
    ret = False
    for i in range(len(a)):
        if np.all(b == a[i]):
            ret = True; break
    
    return ret

In [ ]:
@njit
def isin_val(a, b):
    
    ret = False
    for i in range(len(a)):
        if b == a[i]:
            ret = True; break
    
    return ret

In [ ]:
@njit
def exterior_outline_pair_maker(cells_group, size):
    
    exterior_outline_pair = [[0,0,0]][:0]
    for cells_group_i in cells_group:
        for j in [(cells_group_i + offset) % np.array([size, size, size]) for offset in offsets]:
            if not isin(cells_group, j):
                if len(exterior_outline_pair) == 0:
                    exterior_outline_pair.append(list(j))
                else:
                    if not isin(np.array(exterior_outline_pair), j) and not isin(cells_group, j):
                        exterior_outline_pair.append(list(j))
    
    return np.array(exterior_outline_pair)

In [ ]:
@njit
def check_neighbors_cube(fg, isolated_ij, size, stop_after_2=False):
    
    '''
    Finds the neighbors of a cell inside the data cube, where a neighbor is defined as a cell already found to 
        be a void (thus values >=0, see description inside the Finder code).
    
    Note: The np.clip() function and the summation of arrays require arrays (obviously). So the input must also
        be arrays (inside the @njit function).
    '''
    

    # Coordinates of all the cell's neighbors (including over the edge)
    neighbor_coords = exterior_outline_pair_maker(isolated_ij, size)
    neighbor_values = [0][:0]
    for nc in range(len(neighbor_coords)):
        if stop_after_2 and len(neighbor_coords) == 2: break
        [i,j,k] = neighbor_coords[nc]
        nc_val = fg[i][j][k]
        if nc_val >= 0:
            if len(neighbor_values) == 0:
                neighbor_values.append(nc_val)
            else:
                if not isin_val(neighbor_values, nc_val):
                    neighbor_values.append(nc_val)
    
    return neighbor_values

---

In [ ]:
@njit
def loop_isolated(fg, isolated_i, od_connections_all_val, ud, od, void_index_od, MK, size):
    
    '''
    Only looking at the isolated ("alone") cells.
    Because of that, we can instantly modify the filled_grid.
    '''
    
    isolated_i = [list(i) for i in isolated_i]

    
    try_it_all_again = True
    while try_it_all_again and (len(isolated_i) != 0):
        try_it_all_again = False
        isolated_i_new = [[0,0,0]][:0]
        for isolated_ij in isolated_i:
            i0,j0,k0 = isolated_ij
    
            # Find all of the cell's non-wall neighbors in the filled_grid (including over the edge, ofc).
            void_vals = check_neighbors_cube(fg, np.array([isolated_ij]), size, stop_after_2=ud)
            no_voids  = len(void_vals)

            
            if no_voids == 1:
                # Only touches one void (from prev levels, since it's isolated) (and maybe some walls) -> becomes that void.
                # In MK1, it may touch a void from this level when we try_it_all_again.
                try_it_all_again = True
                fg[i0][j0][k0] = void_vals[0]

            elif no_voids == 0:
                if MK == "MK2":
                    # If we use MK2 and an isolated cell does not touch any void, it was isolated
                    #     from a previous level... must become a wall.
                    # If we use MK1, we come back to this case after we can't try_it_all_again.
                    fg[i0][j0][k0] = -2
                else:
                    isolated_i_new.append(isolated_ij)
                    
    
            # Touches multiple voids and maybe walls (from prev levels, since it's isolated in MK2) -> becomes a wall.
            elif no_voids >= 2:
                
                if ud:
                    # If we are ud, combine all the voids it neighbors into one, keeping the index of the first (not
                    #     that it matters)....
                    try_it_all_again = True
                    vv0 = void_vals[0]
                    fg[i0][j0][k0] = vv0
                    # and replace the others neighboring it in this underdensity wall with its index.
                    for vv in void_vals[1:]:
                        for [i00,j00,k00] in np.argwhere(fg == vv):
                            fg[i00][j00][k00] = vv0
                    # Since we are ud, we can't have any walls yet.... so we don't need to worry about removing 
                    #     any walls betweent the merged voids.
                
                else:
                    if not od:
                        # If we are not od, we don't need to make pairs for the merged voids.
                        fg[i0][j0][k0] = -2
                    
                    else:
                        # But if we are... we do.
                        regular_neighbors = -1; regular_neighbors_len = 0; od_neighbors = [0][:0]
                        for iii in void_vals:
                            if   iii  < void_index_od:
                                regular_neighbors = iii; regular_neighbors_len += 1
                            elif iii >= void_index_od:
                                od_neighbors.append(iii)
                        
                        if   regular_neighbors_len >= 2:
                            # If there are at least two values that are non-od voids, it is a wall.
                            fg[i0][j0][k0] = -2
                        
                        elif regular_neighbors_len == 1:
                            # If there's just one, then it we glue this cell to it, since later it would
                            #     become that regardless due to lack of competition on it, but also because
                            #     it is more advantageour computationally to have more smaller pairs rather
                            #     than few larger ones... this regular void will spread into all og's regardless.
                            try_it_all_again = True
                            fg[i0][j0][k0] = regular_neighbors
                            
                            for odn in od_neighbors:
                                if od_connections_all_val[odn-void_index_od] == -1:
                                    od_connections_all_val[odn-void_index_od] = regular_neighbors
                                else:
                                    od_connections_all_val[odn-void_index_od] = -2

                        else:
                            # If there are only od voids, we add it to the first of then, and if there are
                            #     more, merge them. Here, we don't have any fake walls (between merging voids), 
                            #     ofc... this would've been the first.
                            try_it_all_again = True
                            odn0 = od_neighbors[0]
                            fg[i0][j0][k0] = odn0
                            
                            for odn in od_neighbors[1:]:
                                odcav0 = od_connections_all_val[odn0-void_index_od]
                                odcav  = od_connections_all_val[odn -void_index_od]
                                od_connections_all_val[odn-void_index_od] = -1
                                if odcav0 != -2:
                                    if   odcav == -2:
                                        od_connections_all_val[odn0-void_index_od] = -2
                                    elif odcav >=  0:
                                        if odcav0 == -1:
                                            od_connections_all_val[odn0-void_index_od] = odcav
                                        else:
                                            od_connections_all_val[odn0-void_index_od] = -2
                    
        isolated_i = isolated_i_new   # We can copy it like this since the next step in the loop is to redefine isolated_i_new.


    # If we still have some unsolved cells...
    for ii in range(len(isolated_i)):
        # it means they were isolated from a previous level... nothing we can do... make them walls.
        i,j,k = isolated_i[ii]
        fg[i][j][k] = -2

In [ ]:
@njit
def loop_isolated_leftovers(fg, isolated_i, size):

    
    isolated_i = [list(i) for i in isolated_i]

    
    try_it_all_again = True
    while try_it_all_again and (len(isolated_i) != 0):
        try_it_all_again = False
        isolated_i_new = [[0,0,0]][:0]
        for isolated_ij in isolated_i:
            i0,j0,k0 = isolated_ij
    
            # Find all of the cell's non-wall neighbors in the filled_grid (including over the edge, ofc).
            void_vals = check_neighbors_cube(fg, np.array([isolated_ij]), size, stop_after_2=ud)
            no_voids  = len(void_vals)

            
            if no_voids == 1:
                fg[i0][j0][k0] = void_vals[0]
                try_it_all_again = True

            elif no_voids == 0:
                isolated_i_new.append(isolated_ij)
                    
            elif no_voids >= 2:
                fg[i0][j0][k0] = -2
                    
        isolated_i = isolated_i_new   # We can copy it like this since the next step in the loop is to redefine isolated_i_new.
    
    for ii in range(len(isolated_i)):
        i,j,k = isolated_i[ii]
        fg[i][j][k] = -2

---
---
---

In [ ]:
@njit
def check_neighbors_direct(coords, vals, coord_0, size):
    
    coord_diff = np.abs(coords - coord_0)
    
    condition = ((coord_diff[:, 0] <= 1) | (coord_diff[:, 0] == size-1)) & ((coord_diff[:, 1] <= 1) | (coord_diff[:, 1] == size-1)) & ((coord_diff[:, 2] <= 1) | (coord_diff[:, 2] == size-1))
    
    return vals[condition]

In [ ]:
@njit
def check_walls_direct(coords, coord_0, size):
    
    coord_diff = np.abs(coords - coord_0)

    condition = ((coord_diff[:, 0] <= 1) | (coord_diff[:, 0] == size-1)) & ((coord_diff[:, 1] <= 1) | (coord_diff[:, 1] == size-1)) & ((coord_diff[:, 2] <= 1) | (coord_diff[:, 2] == size-1))

    return len(coords[condition]) != 0

In [ ]:
@njit
def check_ghost_bridge(coords, coord_0, fg, set_val, use_od, void_index_od, size):

    coord_diff = np.abs(coords - coord_0)

    condition = ((coord_diff[:, 0] <= 1) | (coord_diff[:, 0] == size-1)) & ((coord_diff[:, 1] <= 1) | (coord_diff[:, 1] == size-1)) & ((coord_diff[:, 2] <= 1) | (coord_diff[:, 2] == size-1))
    
    coords = coords[condition]

    found_one = False
    for coord in coords:
        neighbor_coords = (coord + offsets) % np.array([size, size, size])
        neighbor_values = [0][:0]
        
        for [i0,j0,k0] in neighbor_coords:
            fg0 = fg[i0][j0][k0]
            if (fg0 != set_val) and (fg0 >= 0) and (fg0 < void_index_od if use_od else True):
                found_one = True; break

        if found_one: break

    return found_one

In [ ]:
@njit
def find_connected_groups(coords, size):

    # grouped coordinates
    visited = np.zeros(len(coords), dtype=np.bool_)
    
    # groups
    max_groups  = len(coords)
    groups      = np.zeros((max_groups, len(coords), 3), dtype=np.int32)
    group_sizes = np.zeros( max_groups,                  dtype=np.int32)
    group_count = 0
    
    def are_neighbors(coord1, coord2):
        for offset in offsets:
            if np.array_equal((coord1 + offset) % np.array([size,size,size]), coord2): return True
        return False
    
    def find_group(start_idx):
        stack = [start_idx]
        group_idx = group_count
        group_size = 0
        
        while stack:
            idx = stack.pop()
            if visited[idx]: continue
            visited[idx] = True
            
            # Add to the current group
            groups[group_idx, group_size] = coords[idx]
            group_size += 1
            
            for j in range(len(coords)):
                if not visited[j] and are_neighbors(coords[idx], coords[j]): stack.append(j)
        
        return group_size
    
    for i in range(len(coords)):
        if not visited[i]:
            group_size = find_group(i)
            group_sizes[group_count] = group_size
            group_count += 1
    
    groups = groups[:group_count]
    group_sizes = group_sizes[:group_count]
    
    return [[j for j in groups[i, :group_sizes[i]]] for i in range(len(group_sizes))]

---

In [ ]:
@njit
def factorial(n):
    
    result = 1
    for i in range(2, n + 1):
        result *= i
    
    return result

In [ ]:
@njit
def permutation(pool, perm_index):
    
    '''
    Creates a unique permutation of the elements inside a list, which is replicable thoough the perm_index 
        parameter.
    '''
    
    pool = [list(i) for i in pool]
    perm_index -= 1
    n = len(pool)
    
    ran_out_of_perm = False
    if perm_index+1 > factorial(n):
        permt = pool
        ran_out_of_perm = True
        
    if not ran_out_of_perm:
        indices = list(range(n))
        cycles = list(range(n, 0, -1))

        for i0 in range(perm_index):
            for i in range(n-1, -1, -1):
                cycles[i] -= 1
                if cycles[i] == 0:
                    indices[i:] = indices[i+1:] + indices[i:i+1]
                    cycles[i] = n - i
                else:
                    j = cycles[i]
                    indices[i], indices[-j] = indices[-j], indices[i]
                    break
    
    permt = [pool[i] for i in indices[:n]]
    
    return permt, ran_out_of_perm

---

In [ ]:
@njit
def loop_pairs(fg, pairs, od_connections_all_val, size, use_od, void_index_od, times_we_tried_max=100):
    
    # In case we try a different permuation... we remove all the ghosts and super ghosts from the grid.
    all_g_v_coord = [[0,0,0]][:0]; all_sg_v_coord = [[0,0,0]][:0]  
    all_g_w_coord = [[0,0,0]][:0]; all_sg_w_coord = [[0,0,0]][:0]

    pairs = [list(i) for i in pairs]
    
    times_we_tried = 0; try_it_all_again = True
    while try_it_all_again:
        try_it_all_again = False; times_we_tried += 1
        times_end = times_we_tried == times_we_tried_max
        
        len_pairs_original = len(pairs)
        less_than_4_left = len_pairs_original <= 3
        
        ##########################################################################################
        
        
        # Gotta define them here so numba won't be mad at us.
        p_c_v_coord = [[0,0,0]][:0]; p_g_v_coord = [[0,0,0]][:0]; p_sg_v_coord = [[0,0,0]][:0]
        p_c_v_vals  = [ 0][:0];      p_g_v_vals  = [ 0][:0];      p_sg_v_vals  = [ 0][:0]
        
        p_c_w_coord = [[0,0,0]][:0]; p_g_w_coord = [[0,0,0]][:0]; p_sg_w_coord = [[0,0,0]][:0]
    
        found_one_p_c_v = False; found_one_p_g_v = False; found_one_p_sg_v = False
        found_one_p_c_w = False; found_one_p_g_w = False; found_one_p_sg_w = False
    
    
        # We try until all the elemnts either touched a void or we ran them once more and none did.
        try_again = True; run = -1
        while try_again and (len(pairs) != 0):
            try_again = False; run += 1
            
            c_c_v_coord = [[0,0,0]][:0]; c_g_v_coord = [[0,0,0]][:0]; c_sg_v_coord = [[0,0,0]][:0]
            c_c_v_vals  = [ 0][:0];      c_g_v_vals  = [ 0][:0];      c_sg_v_vals  = [ 0][:0]
    
            c_c_w_coord = [[0,0,0]][:0]; c_g_w_coord = [[0,0,0]][:0]; c_sg_w_coord = [[0,0,0]][:0]
    
            found_one_c_c_v = False; found_one_c_g_v = False; found_one_c_sg_v = False
            found_one_c_c_w = False; found_one_c_g_w = False; found_one_c_sg_w = False
            
            no_pairs_deleted = 0
            for i0, pair_0 in enumerate(pairs):
                certain = False; ghost = False; superghost = False; void = False; wall = False
                t_g_w = False; t_sg_w = False; t_sgg_w = False
                use__p_g_v = False; cant_be_super = True
                set_val = -1
    
    
    
                #############################################
                
                if run == 0:
                    vals_fg = check_neighbors_cube(fg, np.array([pair_0]), size, stop_after_2=True)
                    if not use_od: vals_fg = [i for i in vals_fg if i < void_index_od]
                    len_fg = len(vals_fg)
    
                    if len_fg == 2:
                        certain = True; wall = True
                    
                    elif len_fg == 1:
                        set_val = vals_fg[0]
                        if found_one_c_c_v:
                            vals_c_c_v = check_neighbors_direct(np.array(c_c_v_coord), np.array(c_c_v_vals), np.array(pair_0), size)
                            len_c_c_v = len(vals_c_c_v)
                            
                            if (len_c_c_v == 2) or ((len_c_c_v == 1) and vals_c_c_v[0] != set_val):
                                certain = True; wall = True
    
                
        
                else:
                    if found_one_p_c_v:
                        vals_p_c_v = check_neighbors_direct(np.array(p_c_v_coord), np.array(p_c_v_vals), np.array(pair_0), size)
                        len_p_c_v = len(vals_p_c_v)
                        use__p_g_v = True
                        
                        if len_p_c_v == 2:
                            certain = True; wall = True
                        
                        elif len_p_c_v == 1:
                            set_val = vals_p_c_v[0]
                            if found_one_c_c_v:
                                vals_c_c_v = check_neighbors_direct(np.array(c_c_v_coord), np.array(c_c_v_vals), np.array(pair_0), size)
                                len_c_c_v = len(vals_c_c_v)
            
                                if (len_c_c_v == 2) or ((len_c_c_v == 1) and (vals_c_c_v[0] != set_val)):
                                    certain = True; wall = True
            
                            if (not certain) and (found_one_p_sg_v or found_one_c_sg_v):
                                vals_pc_sg_v = check_neighbors_direct(np.array(p_sg_v_coord+c_sg_v_coord), np.array(p_sg_v_vals+c_sg_v_vals), np.array(pair_0), size)
                                len_pc_sg_v = len(vals_pc_sg_v)
        
                                if (len_pc_sg_v == 2) or ((len_pc_sg_v == 1) and (vals_pc_sg_v[0] != set_val)):
                                    ghost = True; wall = True
    
                                elif (len_pc_sg_v == 1) and (vals_pc_sg_v[0] == set_val):
                                    ghost = True
                                            
                                    
                                
                    elif found_one_p_g_v:
                        vals_p_g_v = check_neighbors_direct(np.array(p_g_v_coord), np.array(p_g_v_vals), np.array(pair_0), size)
                        len_p_g_v = len(vals_p_g_v)
                        
                        if len_p_g_v == 2:
                            wall = True
    
                            superghost = True
                            if found_one_c_c_v:
                                vals_c_c_v = check_neighbors_direct(np.array(c_c_v_coord), np.array(c_c_v_vals), np.array(pair_0), size)
                                len_c_c_v = len(vals_c_c_v)
            
                                if   (len_c_c_v == 2): certain = True; superghost = False
                                elif (len_c_c_v == 1): ghost   = True; superghost = False
                                
    
                        
                        elif len_p_g_v == 1:
                            set_val = vals_p_g_v[0]
                            found_one_neighb_c_c_v = False
                            
                            if found_one_c_c_v:
                                vals_c_c_v = check_neighbors_direct(np.array(c_c_v_coord), np.array(c_c_v_vals), np.array(pair_0), size)
                                len_c_c_v = len(vals_c_c_v)
    
                                if (len_c_c_v != 0):
                                    found_one_neighb_c_c_v = True
            
                                if (len_c_c_v == 2):
                                    certain = True; wall = True
                                
                                elif ((len_c_c_v == 1) and (vals_c_c_v[0] != set_val)):
                                    ghost = True; wall = True
                                
                                elif ((len_c_c_v == 1) and (vals_c_c_v[0] == set_val)) and (found_one_p_sg_v or found_one_c_sg_v):
                                    vals_pc_sg_v = check_neighbors_direct(np.array(p_sg_v_coord+c_sg_v_coord), np.array(p_sg_v_vals+c_sg_v_vals), np.array(pair_0), size)
                                    len_pc_sg_v = len(vals_pc_sg_v)
                
                                    if (len_pc_sg_v == 2) or ((len_pc_sg_v == 1) and (vals_pc_sg_v[0] != set_val)):
                                        ghost = True; wall = True
            
                                    elif (len_pc_sg_v == 1) and (vals_pc_sg_v[0] == set_val):
                                        ghost = True
        
                            # If we either found no c_c_v or we did but none touch this cell...
                            if (not found_one_c_c_v) or (not found_one_neighb_c_c_v):
                                cant_be_super = False
                                
                                if (found_one_p_sg_v or found_one_c_sg_v):
                                    vals_pc_sg_v = check_neighbors_direct(np.array(p_sg_v_coord+c_sg_v_coord), np.array(p_sg_v_vals+c_sg_v_vals), np.array(pair_0), size)
                                    len_pc_sg_v  = len(vals_pc_sg_v)
            
                                    if (len_pc_sg_v == 2) or ((len_pc_sg_v == 1) and (vals_pc_sg_v[0] != set_val)):
                                        superghost = True; wall = True
        
                                    elif (len_pc_sg_v == 1) and (vals_pc_sg_v[0] == set_val):
                                        superghost = True
                                        
                                        
    
                                    
    
                    elif found_one_p_sg_v:
                        vals_p_sg_v = check_neighbors_direct(np.array(p_sg_v_coord), np.array(p_sg_v_vals), np.array(pair_0), size)
                        len_p_sg_v = len(vals_p_sg_v)
                        
                        if len_p_sg_v == 2:
                            wall = True
    
                            superghost = True   # see if it remains after below
                            if found_one_c_c_v:
                                vals_c_c_v = check_neighbors_direct(np.array(c_c_v_coord), np.array(c_c_v_vals), np.array(pair_0), size)
                                len_c_c_v = len(vals_c_c_v)
            
                                if   (len_c_c_v == 2): certain = True; superghost = False
                                elif (len_c_c_v == 1): ghost = True; superghost = False
                            
    
                        elif len_p_sg_v == 1:
                            set_val = vals_p_sg_v[0]
                            superghost = True   # see if it remains after below
                            if found_one_c_c_v:
                                vals_c_c_v = check_neighbors_direct(np.array(c_c_v_coord), np.array(c_c_v_vals), np.array(pair_0), size)
                                len_c_c_v = len(vals_c_c_v)
            
                                if (len_c_c_v == 2):
                                    certain = True; wall = True; superghost = False
                                
                                elif ((len_c_c_v == 1) and (vals_c_c_v[0] != set_val)):
                                    ghost = True; wall = True; superghost = False
                                
                                elif ((len_c_c_v == 1) and (vals_c_c_v[0] == set_val)):
                                    ghost = True; superghost = False
    
    
                            if (not wall):
                                if found_one_c_sg_v:
                                    vals_c_sg_v = check_neighbors_direct(np.array(c_sg_v_coord), np.array(c_sg_v_vals), np.array(pair_0), size)
                                    len_c_sg_v = len(vals_c_sg_v)
    
                                    if (len_c_sg_v == 2) or ((len_c_sg_v == 1) and (vals_c_sg_v[0] != set_val)):
                                        wall = True
    
                
                
                ######################################################################################
    
    
                
                # If we are set to have found a void or a wall...
                if set_val != -1:
    
                    ##### BLUE
                    # If we are still not sure what type of structure this cell is... and we are not run run==0 (simply because
                    #     in this first run we can't have any super ghosts)...
                    # A new type of check appears: our cell might neighbor a super ghost wall, which can turn it into a ghost or
                    #     even a super ghost if it has not (and thus will not) touch a certain void.
                    # In the later case, if it does not touch a super ghost wall, then it must be a ghost.
                    if (not run == 0) and (not certain) and (not ghost) and (not superghost):
                        
                        coord_to_check  = p_sg_w_coord + c_sg_w_coord
    
                        # Has touched a certain void... can't be a super ghost.
                        if cant_be_super:
                            if len(coord_to_check) != 0:
                                if check_walls_direct(np.array(coord_to_check), np.array(pair_0), size):
                                    ghost = True
                        
                        # Has not touched a certain void... can be a super ghost, but at least must be a ghost.
                        else:
                            ghost = True
                            if len(coord_to_check) != 0:
                                if check_walls_direct(np.array(coord_to_check), np.array(pair_0), size):
                                    superghost = True
                                    ghost = False
                            
                                
                    
                    ##### PURPLE
                    # If we tried all the above checks...
                    # and still haven't set on wether we find a void or a wall...
                    # we must check ghost voids.
                    if (not void) and (not wall):
                        
                        void = True
                        # If we are guaranteed a different neighboring void, this cell is a wall... otherwise it must be a void.
                        if found_one_p_g_v or found_one_c_g_v:
                        
                            # Check previous (if not included in our above search already) and current voids.
                            coord_to_check = c_g_v_coord; vals_to_check = c_g_v_vals
                            if use__p_g_v: coord_to_check += p_g_v_coord; vals_to_check += p_g_v_vals

                            if len(coord_to_check) != 0:
                                vals_checked = check_neighbors_direct(np.array(coord_to_check), np.array(vals_to_check), np.array(pair_0), size)
                                len_vals_checked = len(vals_checked)
                            
                                if (len_vals_checked == 2) or ((len_vals_checked == 1) and (vals_checked[0] != set_val)):
                                    wall = True
                                    void = False
                                    
                                    # If we arrived to this point, we cannot have a certain wall...
                                    # but in some cases, we don't even know if this is a ghost or a superghost wall.
                                    # Then, since super ghosts require certainty (we would have had to have already touched a super ghost),
                                    #     we know that this must be a ghost.
                                    if (not superghost): ghost = True
    
    
                    ##### GREEN
                    # If we are set to have found a void and we're still not sure what type it can either be a certain or a ghost one.
                    if (void) and (not certain) and (not ghost) and (not superghost):
    
                        # Check all cells that might form a bridge to a previous peel void:
                        # (all remaining pairs... any we already ran over would have already given that bridge in the previous
                        #     checks) + (previous and current walls that are either ghosts or superghosts... so that they could
                        #     have returned a void in another permuation)
                        coord_to_check = pairs[i0 - no_pairs_deleted + 1:] + p_g_w_coord + p_sg_w_coord + c_g_w_coord + c_sg_w_coord
    
                        certain = True
                        # If we have such cells to check...
                        if len(coord_to_check) != 0:
                            # and they do form a bridge...
                            if check_ghost_bridge(np.array(coord_to_check), np.array(pair_0), fg, set_val, use_od, void_index_od, size):
                                ghost = True
                                certain = False
                    
                
                ######################################################################################
    
                
                if void:
                    try_again = True
                if void or wall:
                    i0_del = i0 - no_pairs_deleted; no_pairs_deleted += 1
                    pairs = pairs[:i0_del] + pairs[i0_del+1:]
                
                
                if certain:
                    if wall:
                        found_one_c_c_w = True
                        c_c_w_coord.append(pair_0)
                    else:
                        found_one_c_c_v = True
                        c_c_v_coord.append(pair_0); c_c_v_vals.append( set_val)
    
                if ghost:
                    if wall:
                        found_one_c_g_w = True
                        c_g_w_coord.append(  pair_0)
                        if not times_end: all_g_w_coord.append(pair_0)
                    else:
                        found_one_c_g_v = True
                        c_g_v_coord.append(pair_0); c_g_v_vals.append( set_val)
                        if not times_end: all_g_v_coord.append(pair_0)
    
                if superghost:
                    if wall:
                        found_one_c_sg_w = True
                        c_sg_w_coord.append(pair_0)
                        if not times_end: all_sg_w_coord.append(pair_0)
                    else:
                        found_one_c_sg_v = True
                        c_sg_v_coord.append(pair_0); c_sg_v_vals.append(set_val)
                        if not times_end: all_sg_v_coord.append(pair_0)



            ##########################################################################################

            
            if found_one_c_c_v:  found_one_p_c_v  = True
            else:                found_one_p_c_v  = False
            if found_one_c_c_w:  found_one_p_c_w  = True
            else:                found_one_p_c_w  = False
            if found_one_c_g_v:  found_one_p_g_v  = True
            else:                found_one_p_g_v  = False
            if found_one_c_g_w:  found_one_p_g_w  = True
            else:                found_one_p_g_w  = False
            if found_one_c_sg_v: found_one_p_sg_v = True
            else:                found_one_p_sg_v = False
            if found_one_c_sg_w: found_one_p_sg_w = True
            else:                found_one_p_sg_w = False
            
            p_c_v_coord = c_c_v_coord.copy(); p_g_v_coord = c_g_v_coord.copy(); p_sg_v_coord = c_sg_v_coord.copy()
            p_c_v_vals  = c_c_v_vals.copy();  p_g_v_vals  = c_g_v_vals.copy();  p_sg_v_vals  = c_sg_v_vals.copy()
            p_c_w_coord = c_c_w_coord.copy(); p_g_w_coord = c_g_w_coord.copy(); p_sg_w_coord = c_sg_w_coord.copy()
            
            # To make the check_cross_bridge() work, we need at the end of each peel to add to fg all the cells we just found.
            if found_one_c_c_v:
                for [ip,jp,kp], val_p in zip(c_c_v_coord, c_c_v_vals):   fg[ip][jp][kp] = val_p
            if found_one_c_g_v:
                for [ip,jp,kp], val_p in zip(c_g_v_coord, c_g_v_vals):   fg[ip][jp][kp] = val_p
            if found_one_c_sg_v:
                for [ip,jp,kp], val_p in zip(c_sg_v_coord, c_sg_v_vals): fg[ip][jp][kp] = val_p
            if found_one_c_c_w:
                for [ip,jp,kp] in c_c_w_coord:  fg[ip][jp][kp] = -2
            if found_one_c_g_w:
                for [ip,jp,kp] in c_g_w_coord:  fg[ip][jp][kp] = -2
            if found_one_c_sg_w:
                for [ip,jp,kp] in c_sg_w_coord: fg[ip][jp][kp] = -2
                    
            
        ##############################################################################################
        

        # Now it may be that we still have some cells that are only connected to an od void.
        len_pairs_modified = len(pairs)
        if len_pairs_modified != 0:
            
            if less_than_4_left or not ((times_we_tried < times_we_tried_max) and (found_one_p_g_v or found_one_p_g_w or found_one_p_sg_v or found_one_p_sg_w)):
                # if == 1  --  It was isolated from the start... nothing we can do... must be a wall.
                # if == 2 or == 3  --  The others (or all) tried to be a bridge for the remaining, but couldn't. The remaining are walls.
                # Same if we tried our best and can't try_it_all_again either because we ran out of attempts
                #     or we didn't find any ghosts or superghosts (if there are no ghosts, then there is no way
                #     to un-isolate the remaining pairs).
                # So if we don't find some connections to outside od voids, these cells are walls: no
                #     more try_it_all_again.

                if not use_od:
                    # If we are not at the od level or we are running an od as a pair, we must make the remainder of this pair walls... it was isolated.
                    for [ip,jp,kp] in pairs: fg[ip][jp][kp] = -2
                
                else:
                    # We first split these remaining cells into the pairs they may form and check them individually.
                    fcg = find_connected_groups(np.asarray(pairs), size)
                    for leftovers_pair in fcg:
                        exterior_outline_pair = exterior_outline_pair_maker(leftovers_pair, size)
                        exterior_outline_pair_vals_od_unique = [i for i in check_neighbors_cube(fg, exterior_outline_pair, size, stop_after_2=False) if i >= void_index_od]
                        
                        if len(exterior_outline_pair_vals_od_unique) == 0:
                            # If it is truly isolated and since we can't try_it_all_again, all remaining cells are walls.
                            for [ip,jp,kp] in leftovers_pair: fg[ip][jp][kp] = -2
                        
                        elif   len(exterior_outline_pair_vals_od_unique) == 1:
                            # If it touches one od void, it is part of it.
                            eopvudu0 = exterior_outline_pair_vals_od_unique[0]
                            for [ip,jp,kp] in leftovers_pair: fg[ip][jp][kp] = eopvudu0
                        
                        else:
                            # If it touches multiple od voids, merge them and add this cell to the first od.
                            # Also, add this lefotver's connections to the first od.
                            eopvudu0 = exterior_outline_pair_vals_od_unique[0]
                            for [ip,jp,kp] in leftovers_pair: fg[ip][jp][kp] = eopvudu0
                            for eopvudu in exterior_outline_pair_vals_od_unique[1:]:

                                for [i00,j00,k00] in np.argwhere(fg == eopvudu):
                                    fg[i00][j00][k00] = eopvudu0
                                    odcav0 = od_connections_all_val[eopvudu0-void_index_od]
                                    odcav  = od_connections_all_val[eopvudu -void_index_od]
                                    od_connections_all_val[eopvudu -void_index_od] = -1
                                    if odcav0 != -2:
                                        if   odcav == -2:
                                            od_connections_all_val[eopvudu0-void_index_od] = -2
                                        elif odcav >=  0:
                                            if odcav0 == -1:
                                                od_connections_all_val[eopvudu0-void_index_od] = odcav
                                            else:
                                                od_connections_all_val[eopvudu0-void_index_od] = -2
                            
                    
            
            else:
                # Same thing, but now we try_it_all_again if even one cell remains unfilled, which happens if even
                #     one leftovers_pair does not touch an (outside) od void (worthwile prior check).
                all_leftovers_touch_od = False
                if use_od:
                    all_leftovers_touch_od = True
                    exterior_outline_pair_all = [[[0,0,0]]][:0]
                    exterior_outline_pair_vals_od_unique_all = [[0]][:0]
                    fcg = find_connected_groups(np.asarray(pairs), size)
                    for leftovers_pair in fcg:
                        exterior_outline_pair = exterior_outline_pair_maker(leftovers_pair, size)
                        exterior_outline_pair_vals_od_unique = [i for i in check_neighbors_cube(fg, exterior_outline_pair, size, stop_after_2=False) if i >= void_index_od]
                        
                        if len(exterior_outline_pair_vals_od_unique) == 0:
                            all_leftovers_touch_od = False; break
                        else:
                            exterior_outline_pair_all.append([list(i) for i in exterior_outline_pair])
                            exterior_outline_pair_vals_od_unique_all.append(exterior_outline_pair_vals_od_unique)
    
                    # If they all do touch an outside od void, we simply sort them as such and we're done.
                    if all_leftovers_touch_od:
                        for lp_i, leftovers_pair in enumerate(fcg):
                            exterior_outline_pair = exterior_outline_pair_all[lp_i]
                            exterior_outline_pair_vals_od_unique = exterior_outline_pair_vals_od_unique_all[lp_i]
                            eopvudu0 = exterior_outline_pair_vals_od_unique[0]
                            
                            # If it touches one od void, it is part of it.
                            if   len(exterior_outline_pair_vals_od_unique) == 1:
                                for [ip,jp,kp] in leftovers_pair: fg[ip][jp][kp] = eopvudu0

                            # If it touches multiple od voids, merge them and add it to that.
                            # Also, add the connections to the list.
                            elif len(exterior_outline_pair_vals_od_unique) >= 2:
                                for [ip,jp,kp] in leftovers_pair: fg[ip][jp][kp] = eopvudu0
                                for eopvudu in exterior_outline_pair_vals_od_unique[1:]:
                                    for [i00,j00,k00] in np.argwhere(fg == eopvudu):
                                        fg[i00][j00][k00] = eopvudu0
                                        odcav0 = od_connections_all_val[eopvudu0-void_index_od]
                                        odcav  = od_connections_all_val[eopvudu -void_index_od]
                                        od_connections_all_val[eopvudu -void_index_od] = -1
                                        if odcav0 != -2:
                                            if   odcav == -2:
                                                od_connections_all_val[eopvudu0-void_index_od] = -2
                                            elif odcav >=  0:
                                                if odcav0 == -1:
                                                    od_connections_all_val[eopvudu0-void_index_od] = odcav
                                                else:
                                                    od_connections_all_val[eopvudu0-void_index_od] = -2

                                    
                # If even one doesn't or we don't use od, we try_it_all_again.
                if not all_leftovers_touch_od:
                    try_it_all_again = True
                    
                    if len_pairs_modified < len_pairs_original:
                        # If we cut-down some elements, we assume this was a good run... not worth going back though all permutations
                        #     in this scenario.
                        have_to_do_a_permutation = False; permutations_done = 1

                    # Basically and "else:"
                    if len_pairs_modified == len_pairs_original:
                        # If we haven't advanced, try a different permutation (as long as we tried not too many times and we
                        #     have ghosts to hopefully change).
                        have_to_do_a_permutation = True; permutations_done += 1
                        if permutations_done > factorial(len_pairs_modified): try_it_all_again = False
                            
                            
                        # If we're seat to try a different permutation, we create a new pair lists consisting of all the remaining
                        #     cells and all the ghosts and super ghosts, which we also remove from the grid.
                        pairs_next_run = pairs.copy()
                        if len(all_g_v_coord) != 0:
                            for pair_0 in all_g_v_coord:
                                pairs_next_run.append(pair_0)
                                ip,jp,kp = pair_0
                                fg[ip][jp][kp] = -1
                        if len(all_g_w_coord) != 0:
                            for pair_0 in all_g_w_coord:
                                pairs_next_run.append(pair_0)
                                ip,jp,kp = pair_0
                                fg[ip][jp][kp] = -1
                        if len(all_sg_v_coord) != 0:
                            for pair_0 in all_sg_v_coord:
                                pairs_next_run.append(pair_0)
                                ip,jp,kp = pair_0
                                fg[ip][jp][kp] = -1
                        if len(all_sg_w_coord) != 0:
                            for pair_0 in all_sg_w_coord:
                                pairs_next_run.append(pair_0)
                                ip,jp,kp = pair_0
                                fg[ip][jp][kp] = -1
    
                        
                        pairs, _ = permutation(np.array(pairs_next_run), permutations_done)

---
---
---

In [ ]:
@njit
def thin_walls(fg, size):
    
    '''
    There is, however, one correction ot the data we shall make.

    We have establighed that there might be wall cells which shouldn't be. This might create the thick walls.
    
    Very important: we only thin walls which are of the same or higher level than their void neighbors... otherwise we'd make something that should be a new void inside
    the void in theighbors.
    
    We can partly mitigete this issue by doing the following (remember, much like adding walls, thinning them must also be cone in a layered manner):
        1. Iterate over all the wall cells in this level.
        2. If a cell is a wall and has a single neighbouring void (if it has none, it is fully isolated and we'll fix it in a following round if by then it will only have gained a single void as a neighbor) make that cell part of that void.
        3. Go through the whole level, then repeat until there are no newly set void cells.
        
    Since the cells in the ll_i list are randomly ordered, this is an efficient and theory-sound way to deal with this reverse peeling process.
    '''
    
    
    
    w_inner = [list(i) for i in np.argwhere(fg == -2)]   # isolated walls (completely surrounded by other walls)
    w_removed = [[0,0,0]][:0]
    try_again = True
    no_peels = -1
    
    while len(w_inner) != 0 and try_again:
        try_again = False; no_peels += 1

        w_inner_new = [[0,0,0]][:0]
        
        for i0 in range(len(w_inner)):
            w_inner_i = w_inner[i0]
            [i,j,k] = w_inner_i
            
            # we check if it neighbours at least two voids (otherwise it is a false wall)
            void_vals = check_neighbors_cube(fg, np.array([w_inner_i]), size, stop_after_2=True)
            no_voids = len(void_vals)
            
            # it might not even touch a single void
            # but we can only fix those that do touch one in this round
            if no_voids == 1:
                fg[i][j][k] = void_vals[0]
                w_removed.append(w_inner_i)
                try_again = True
            elif no_voids == 0:
                w_inner_new.append(w_inner_i)
        
        w_inner = w_inner_new.copy()
        
    return w_removed, no_peels

---
---
---